# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities_mywork.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.01,80,8,3.13,NZ,1687492044
1,1,san juan de miraflores,-12.1629,-76.9635,20.29,81,77,4.54,PE,1687492045
2,2,bouna,9.2667,-3.0000,21.77,88,63,2.97,CI,1687492045
3,3,kodiak,57.7900,-152.4072,9.73,93,100,2.06,US,1687492003
4,4,mrakovo,52.7161,56.6244,11.52,66,100,1.44,RU,1687492045


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", #"EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    #scale = 0.5,
    alpha = 0.6,
    color = "City"    
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_df = city_data_df.dropna()
# Drop any rows with null values
# YOUR CODE HERE
city_data_narrow_df = city_data_df.loc[(city_data_df["Max Temp"].between(21, 27) & 
                                       (city_data_df["Wind Speed"] < 4.5) & 
                                       (city_data_df["Cloudiness"] == 0))]
# Display sample data
# YOUR CODE HERE
city_data_narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,prineville,44.2999,-120.8345,24.39,29,0,3.09,US,1687492046
13,13,al badari,26.9926,31.4155,24.08,42,0,3.73,EG,1687492049
139,139,ghat,24.9647,10.1728,26.04,21,0,1.63,LY,1687492088
141,141,salobrena,36.7428,-3.5872,24.15,77,0,1.83,ES,1687492088
187,187,mafraq,32.3333,37.9167,23.30,37,0,4.30,JO,1687492105
207,207,torrox,36.7579,-3.9523,23.54,59,0,1.34,ES,1687492110
221,221,petropavlovsk-kamchatsky,53.0452,158.6483,24.33,38,0,4.00,RU,1687492115
228,228,chebba,35.2372,11.1150,25.69,48,0,2.22,TN,1687492117
262,262,gazi,35.3253,25.0669,25.89,60,0,2.57,GR,1687492127
302,302,remire-montjoly,4.9167,-52.2667,24.02,94,0,0.00,GF,1687492094


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_narrow_df[['City', 'Country','Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['hotel'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,hotel
6,prineville,US,44.2999,-120.8345,29,
13,al badari,EG,26.9926,31.4155,42,
139,ghat,LY,24.9647,10.1728,21,
141,salobrena,ES,36.7428,-3.5872,77,
187,mafraq,JO,32.3333,37.9167,37,
207,torrox,ES,36.7579,-3.9523,59,
221,petropavlovsk-kamchatsky,RU,53.0452,158.6483,38,
228,chebba,TN,35.2372,11.1150,48,
262,gazi,GR,35.3253,25.0669,60,
302,remire-montjoly,GF,4.9167,-52.2667,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    # YOUR CODE HERE
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 20
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    target_city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}" # YOUR CODE HERE
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
prineville - nearest hotel: Best Western
al badari - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
salobrena - nearest hotel: Hostal La Paloma
mafraq - nearest hotel: No hotel found
torrox - nearest hotel: Hotel "La CASA"
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
chebba - nearest hotel: No hotel found
gazi - nearest hotel: Manos Studios Hotel
remire-montjoly - nearest hotel: Complexe Belova
at taj - nearest hotel: No hotel found
kebili - nearest hotel: Hôtel du Autruches
al kharijah - nearest hotel: Kharga Hotel
arish - nearest hotel: سينا
kapuskasing - nearest hotel: Travelodge
prairie grove - nearest hotel: No hotel found
ascension - nearest hotel: Cajun Country Inn
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
tamanrasset - nearest hotel: فندق الأمان
almunecar - nearest hotel: Hostal Altamar
torbali - nearest hotel: Metropolis
bouarfa - nearest hotel: Climat du Maroc مناخ المغرب


,City,Country,Lat,Lng,Humidity,hotel,Hotel Name
6,prineville,US,44.2999,-120.8345,29,,Best Western
13,al badari,EG,26.9926,31.4155,42,,No hotel found
139,ghat,LY,24.9647,10.1728,21,,فندق تاسيلي
141,salobrena,ES,36.7428,-3.5872,77,,Hostal La Paloma
187,mafraq,JO,32.3333,37.9167,37,,No hotel found
207,torrox,ES,36.7579,-3.9523,59,,"Hotel ""La CASA"""
221,petropavlovsk-kamchatsky,RU,53.0452,158.6483,38,,ООО Постоялый двор
228,chebba,TN,35.2372,11.1150,48,,No hotel found
262,gazi,GR,35.3253,25.0669,60,,Manos Studios Hotel
302,remire-montjoly,GF,4.9167,-52.2667,94,,Complexe Belova


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", #"EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    alpha = 0.6,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)